# Restaurant Recommendation wth OpenAI &amp; Langchain
In ths notebook we will progressively develop code that leverages OpenAI to recommend a fancy name for a restaurant, then chain that recommended name to get a menu of items from OpenAI.

## Step1: Setting up OpenAI
As first step, you need to login to OpenAI API website and get an API key. I have stored mine in a file `c:\dev\OpenAIKey.txt`, which is **outside** any Github managed folders. You **don't want to accidentally upload your API key to GitHub**. OpenAI API requires that you set an environment variable `OPENAI_API_KEY` before calling any OpenAI APIs. This is what we'll do next.

In [2]:
# setup the OpenAI API Key (read from file)
OPEN_API_KEY_FILE = r"c:\dev\OpenAIKey.txt"
if not os.path.exists(OPEN_API_KEY_FILE):
    raise FileNotFoundError(f"OpenAI API key file not found at {OPEN_API_KEY_FILE}")

OPEN_API_KEY = None
with open(OPEN_API_KEY_FILE, "r") as f:
    OPEN_API_KEY = f.read().strip()
os.environ["OPENAI_API_KEY"] = OPEN_API_KEY

## Step 2 - calling OpenAI API
There are several Large Language Models (LLMs) that you can use - both commercial (such as OpenAI) and OpenSource (such as Bloom from Hugging Face). In this example we'll use LangChain to orchestrate the calls to OpenAI LLM.

In [3]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [4]:
# Step 1 - create our LLM

llm = OpenAI(temperature=0.6)
# the tempature parameter determines how 'creative' the LLM output is - it goes from 0 to 1, with 1 = most
# creative and 0 = least creative. I usually use a value = 0.6 or 0.7

In [6]:
# Step 2 - ask the llm for 5 fancy names for an Chinse restaurant for example
response = llm.predict(
    "Please suggest a fancy name for a Chinese restaurant. Return 5 comma separated names please"
)
response
# response could be soething like
# "\n\n"Golden Dragon Palace, Jade Phoenix Restaurant, Imperial Palace Cuisine, Fortune's Feast, Dynasty Delight."

".\n\nGolden Dragon Palace, Jade Phoenix Restaurant, Imperial Palace Cuisine, Fortune's Feast, Dynasty Delight."

Wow! That was easy 😃. If you want a different cuisine, say Indian, American, Arabic etc., just replace the word Chinese with your prefered cuisine and ask again.

## Step 3 - templatizing your prompt

Do you see a problem here? The cuisine is _hard coded_ into the request, so each time you want to ask for a different type of cuisine, you have to change the prompt. What if we could _parameterize_ the request? That's where a framework such as `LangChain` comes in. We will use the `langchain.prompts.PromptTemplate` class to parameterize our request. A prompt has to be sent to an LLM, and the result fetched - all this can be chained together with an LLM chain, specifically `langchain.chains.LLMChain` class. Here is the code

In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [18]:
prompt_template_name = PromptTemplate(
    # you can have multiple input variables, hence the list
    input_variables=["cuisine"],
    # this is your prompt - same as above except the cuisine is parameterised with {}
    # similar to a Python f-string
    template="Please suggest a fancy name for a {cuisine} restaurant. Return 5 comma separated names without leading numbers.",
    verbose=True,
)
# here is how you format the template
prompt_template_name.format(cuisine="Mexican")

'Please suggest a fancy name for a Mexican restaurant. Return 5 comma separated names without leading numbers.'

In [20]:
# create a chain to send the prompt to your LLM
chain = LLMChain(llm=llm, prompt=prompt_template_name)
response = chain.run(cuisine="Italian")
response

"\n\nGusto's Trattoria, La Dolce Vita Ristorante, La Tavola Cucina, Bella Cucina, Il Buon Cibo."

In [22]:
# let's grab just the first name
rest_name = response.strip().split(",")[0]
rest_name

"Gusto's Trattoria"

## Step 4 - chaining your requests together
Now that we have a recommendation for our restaurant name, let's ask OpenAI to suggest a mennu (10 items) for that restaurant. To do this we'll need to pass the request from the first prompt to the next prompt.
One way to do so is make separate back-to-back requests, like this.

In [23]:
prompt_template_menu = PromptTemplate(
    # you can have multiple input variables, hence the list
    input_variables=["restaurant_name"],
    # this is your prompt - same as above except the cuisine is parameterised with {}
    # similar to a Python f-string
    template="Please suggest a menu for {restaurant_name}. Suggest 10 items as a numbered list",
    verbose=True,
)
# here is how you format the template
prompt_template_menu.format(restaurant_name=rest_name)

"Please suggest a menu for Gusto's Trattoria. Suggest 10 items as a numbered list"

In [26]:
chain2 = LLMChain(llm=llm, prompt=prompt_template_menu)
response2 = chain2.run(restaurant_name=rest_name)
print(response2.strip())

.

1. Spaghetti Carbonara 
2. Lasagna 
3. Margherita Pizza 
4. Eggplant Parmigiana 
5. Grilled Salmon 
6. Chicken Parmigiana 
7. Fettuccine Alfredo 
8. Bruschetta 
9. Caprese Salad 
10. Tiramisu


Yummy 🤤 - no?
But this is not a practical way to make multiple requests - we can let LangChain chain several requests together - that's what Langchain does - it's in the name! 
Let's illustrate this using a **Sequential Chain**, specifically the `SimpleSequentialChain` class, which is a simple prompt chaining mechanism which does essentially what we have done above in 2 separate steps. It chains the input of the 2nd step with the output of the first step and so on...

In [27]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [42]:
# our llm
llm = OpenAI(temperature=0.6)

# prompt template for the restaurant name, given a cuisine
prompt_template_name = PromptTemplate(
    # you can have multiple input variables, hence the list
    input_variables=["cuisine"],
    # this is your prompt - same as above except the cuisine is parameterised with {}
    # similar to a Python f-string
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for it. Only one name please.",
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

# prompt template for menu items given a restaurant name
prompt_template_menu = PromptTemplate(
    # you can have multiple input variables, hence the list
    input_variables=["restaurant_name"],
    # this is your prompt - same as above except the cuisine is parameterised with {}
    # similar to a Python f-string
    template="Please suggest 10-15 menu items for {restaurant_name}. Return as comma separated list",
)

menu_chain = LLMChain(llm=llm, prompt=prompt_template_menu)

In [43]:
# SimpleSequential Chain
from langchain.chains import SimpleSequentialChain

simple_seq_chain = SimpleSequentialChain(
    chains = [name_chain, menu_chain], # NOTE: sequence matters here!!
)
# now execute the chain with variable for FIRST item in chain only! That's it!
response = simple_seq_chain.run("Indian")
print(response)



Murg Tikka, Tandoori Paneer, Tandoori Fish, Chicken Malai Tikka, Seekh Kebab, Vegetable Samosa, Paneer Tikka, Tandoori Naan, Tawa Paneer, Achari Aloo, Tandoori Aloo, Paneer Tikka Masala, Tandoori Roti, Chicken Biryani, Dal Makhani.


## Step 5 - Capturing intermediate output in chains
The `SimpleSequentialChain` helps us stitch prompts together into a chain, so that output from one prompt is fed into another without us having to explicitly program it. 

However, the downside is that we do not get intermediate output. What was the name of the restaurant that the first prompt returned? We don't know! If we cant to capture intermediate output too, then we need to use a different chaining mechanism. Here is where `SequentialChain` comes into play. The following section introduces the `SequentialChain` class. 

**NOTE:** to capture the intermediate output from chain, we need to add an `output key` to the `LLMChain` constructor using the `output_key=XXX` parameter. Otherwise, most the code remains the same as before.

In [44]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [46]:
# our llm
llm = OpenAI(temperature=0.6)

# prompt template for the restaurant name, given a cuisine
prompt_template_name = PromptTemplate(
    # you can have multiple input variables, hence the list
    input_variables=["cuisine"],
    # this is your prompt - same as above except the cuisine is parameterised with {}
    # similar to a Python f-string
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for it. Only one name please.",
)

name_chain = LLMChain(llm=llm, 
    prompt=prompt_template_name,
    # here we add an output_key parameter to capture the output from this prompt
    output_key="restaurant_name",
)

# prompt template for menu items given a restaurant name
prompt_template_menu = PromptTemplate(
    # you can have multiple input variables, hence the list
    input_variables=["restaurant_name"],
    # this is your prompt - same as above except the cuisine is parameterised with {}
    # similar to a Python f-string
    template="Please suggest 10-15 menu items for {restaurant_name}. Return as comma separated list",
)

# chain is the same as before
menu_chain = LLMChain(llm=llm, 
    prompt=prompt_template_menu,
    # here we add an output_key parameter to capture the output from this prompt
    output_key="menu_items",
)

In [48]:
# SimpleSequential Chain
from langchain.chains import SequentialChain

seq_chain = SequentialChain(
    chains = [name_chain, menu_chain], # NOTE: sequence matters here!!
    input_variables = ['cuisine'],     # input variable to first chain in sequence
    # NOTE: here we specify the output variables too
    output_variables = ['restaurant_name', 'menu_items'],
)

# now we execute the chain. Note how the call is a bit different
response = seq_chain({'cuisine':'American'}) 
# NOTE: response from above call is a JSON object with keys for all input variables (just 1 in our case = 'cuisine')
# and for all our output variables ('restaurant_name' and 'menu_items')
# something like this
"""
response = {
    'cuisine': 'American', 
    'restaurant_name': '\n\nThe Gilded Grill.', 
    'menu_items': '\n\nGrilled Salmon, Roasted Chicken, Filet Mignon, Rib-Eye Steak, Prime Rib, Lamb Chops, Baked Trout, Crab Cakes, Lobster Tail, Seafood Paella, Grilled Vegetables, Caesar Salad, Potato Gratin, Steamed Asparagus, Mac and Cheese, Chocolate Lava Cake.'
}
"""
print(response)

{'cuisine': 'American', 'restaurant_name': '\n\nThe Gilded Grill.', 'menu_items': '\n\nGrilled Salmon, Roasted Chicken, Filet Mignon, Rib-Eye Steak, Prime Rib, Lamb Chops, Baked Trout, Crab Cakes, Lobster Tail, Seafood Paella, Grilled Vegetables, Caesar Salad, Potato Gratin, Steamed Asparagus, Mac and Cheese, Chocolate Lava Cake.'}
